In [1]:
# 导入并查看呢数据
import pandas as pd
# 添加数据读取路径
melbourne_file_path = '/Users/fangcheng/sklearn/frist_day/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [3]:
# 强制删除数据中的缺失值，此方法不可先用，首先必须查看数据的基本情况
melbourne_data = melbourne_data.dropna(axis=0)

In [4]:
# 我们将使用点表示法来选择我们想要预测的列，它被称为预测目标。按照惯例，预测目标为y
y = melbourne_data.Price

In [5]:
# 选择特征（手动选择）
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']

In [6]:
# 特征赋值
X = melbourne_data[melbourne_features]

In [7]:
# 查看数据描述
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [8]:
# 数据预览
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


In [9]:
# 构建决策树模型

In [11]:
# 导入sklearn中的决策树模型
from sklearn.tree import DecisionTreeRegressor
# 定义模型。为random_state指定一个数字，以确保每次运行的结果相同
melbourne_model = DecisionTreeRegressor(random_state=1)
# 训练模型
melbourne_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [13]:
print("预测并显示前五列:")
print(X.head())
print("预测如下")
print(melbourne_model.predict(X.head()))

预测并显示前五列:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
预测如下
[1035000. 1465000. 1600000. 1876000. 1636000.]


In [16]:
# 计算平均绝对误差(样本内的误差)
from sklearn.metrics import mean_absolute_error
predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

1115.7467183128902

In [17]:
# 将数据拆分成为训练集和测试集（样本外的误差）
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
melbourne_model.fit(train_X,train_y)
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y,val_predictions))

273518.01872175594


通过上述对样本内和样本外的绝对误差进行计算，发现样本内的误差在1115美元的样子，但是样本外的误差在27万，很显然使用手动选择特征的方式会使模型出现过拟合现象

In [19]:
# 对模型进行优化
# 首先我们可以使用一个实用函数来帮助比较max_leaf_nodes不同值的MAE分数:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [20]:
# 首先使用for循环找出最优参数（用不同的max_leaf_nodes值比较MAE）
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  385696
Max leaf nodes: 50  		 Mean Absolute Error:  279794
Max leaf nodes: 500  		 Mean Absolute Error:  261718
Max leaf nodes: 5000  		 Mean Absolute Error:  271997


在列出的选项中，500是叶子的最优数量。

决策树留留下了一个艰难的决定。有很多叶子的深树会过拟合，因为每个预测都是来自叶子附近的少数房屋的历史数据。但是一棵叶子很少的浅树会表现得很差，因为它无法在原始数据中捕捉到尽可能多的区别。

即使是今天最复杂的建模技术也面临着欠拟合和过拟合之间的紧张关系。但是，许多模型都有聪明的想法，可以带来更好的性能。我们将以随机森林为例。

随机森林使用许多树，它通过平均每个组成树的预测来进行预测。它通常比单一决策树具有更好的预测准确性，并且在默认参数下工作得很好。如果您继续建模，您可以学习更多具有更好性能的模型，但是其中许多模型对获得正确的参数很敏感。

In [21]:
# 构建随机森林模型
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

207226.35743972045


可能还有进一步改进的空间，但这已经比25万的最佳决策树误差有了很大的改进。有一些参数允许你改变随机森林的性能，就像我们改变单个决策树的最大深度一样。但随机森林模型的一个最佳特征是，即使没有这种调整，它们通常也能合理地工作。

什么是深度学习?
近年来，人工智能领域一些最令人印象深刻的进展是在深度学习领域。自然语言翻译、图像识别和游戏都是深度学习模型接近甚至超过人类水平的任务。

那么什么是深度学习呢?深度学习是一种以深度计算堆栈为特征的机器学习方法。这种深度的计算使深度学习模型能够在最具挑战性的现实世界数据集中发现各种复杂的分层模式。

由于其强大的功能和可扩展性，神经网络已经成为深度学习的定义模型。神经网络由神经元组成，每个神经元单独执行一个简单的计算。相反，神经网络的力量来自于这些神经元可以形成的连接的复杂性。